In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import networkx as nx
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')# one time execution
from nltk.corpus import stopwords
nltk.download('punkt') # one time execution
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Overview: 

### 1) First we separate the reviews into different types
### 2) Summarize all the reviews grouped by types using textrank/bert
### 3) Feed the now concise/shortened summaries to our seq2seq model (since our model only accepts ~100-120 tokens as input)
### 4) Obtain short summaries such as "great value", "nutritious", "nice flavor"

In [ ]:
!pip install -q bert-extractive-summarizer

In [5]:
pos_review_cluster = """

Great dog food, my dog has a very sensitive stomach and after spending loads of money on food this one does not affect her stomach and it is a great price!

I only buy Pedigree for my dog Apollo. He loves it. When I adopted him he was 5 months old and u could see his bones, now he just made a year so come January he will get the big dog, dog food.

I have a smallish dog - 15 pounds. She is very picky, but will eat this. The size portion is just right for her and even though I haven't tried it yet, these would really be handy for bringing along for doggie when traveling. Sturdy packaging. I like that more than one flavor is included in the box.

My dog gets half a pouch every morning with a half a pouch of moist in meaty mixed in and has no problems with this food she can't wait to get it of course I have to wake her up first never goes to waste

Fav dog food so far, been having both my dogs eat it for about 5 years now

Just that my dogs loved it

My 15 yr Pom mix loves this high protein food. She acts like a puppy still. So if she likes it, that’s all that matters!

I have a 13 pound shih tzu Yorkie cross who has difficulty with hard kibble. We tried some canned brands which she didn’t like and some drier non kibble type food that also caused her to cough and go off her food (moist and meaty packets). This with the gravy is perfect combo! She loves it and one packet twice a day is helping her maintain her weight.

My dogs love this dog food! We purchased the largest bag we could find on Amazon. We have 4 large dogs and they go through dog food like crazy! The shipping took a little longer but it was totally worth it. I can’t tug at large bags of dog food and having this delivered was perfect!

Same as what I've purchased in other stores. Great item for your fur babies

My dog loves eating the same thing, every day.

My dog likes it!!!!!
"""

In [10]:
neg_review_cluster = """Bought 2 bags of the 18lb Grilled Steak & Vegetables and had to throw them away because my dog had an allergic reaction. Hour after only eating a few bites her face started to swell and turn red. She vomited and was very lethargic. It was well before the expiration date and I didn't see any mold, but other reviews noticed mold. I bought Pedigree before with no issues, but after this incident I'm not buying it anymore.

The dye used in this product stains the carpet. My dog vomited on my carpet and now I have a permanent stain of the rust red kibble. I can't get it out of the carpet so I figure it must be a powerful stain that can't be good for my dog


I have a pit bull and a little terrier mix. My pitbull has allergies and therefore since he was about 6 months old I have dealt with skin issues with him. But the terrier is older, and never a problem. I need to pinch pennies for a few months and got this food. They are fine with it as far as taste, but after about two months I noticed an awful frito like smell in my living room. Then I noticed the little dog was scratching constantly at er ear. Both ears were nasty as could be and draining smelly sticky stuff. And there was crusty yuck too. And it spread to her chest in days. I discovered it was a yeast issue brought on by this cheap food. Well I had no more money for more food, not for three weeks. She had to eat it or go hungry. But I followed instructions online ad got her healed up except one little spot in the grown area, which spread to her whole groin area in a matter of days. This has been so awful fighting this thing. I got her on better food, recommended online, and within a couple weeks she's nearly completely healed and I'll never switch again. I don't know if you can recommend another better brand when downgrading one, so I'll just say, they are on a non-grain popular dog food and doing good. Do not feed this to your dog if your dog has hotspot and/or yeast issues, any skin issues, or if your dog is older, because immune system changes with age. Even if you were unaware your dog had issues while on good food, if there are underlying issues you are not aware of, buy this and you'll become aware!


Dogs loved the taste, but they both developed severe skin reactions within a couple weeks. They had a complete hair loss over the entire belly, and difficult to treat ear infections. They had to go onto Apoquel ($120 per month) to reverse the symptoms and keep them away. Don’t risk your dogs health. Always use good food…always!

Your people will not do as I ask for years now . I have 2 door bells both work I call over and over about this problem.stuff has gotten stolen because they won't ring the door bell they pass right by me I was watching didn't ring it until I came out and ask why didn't u ring the door bell .no answer.i shop all the time from here .that will stop if u can't just ring a bell I have 2

Switched from Rachael Rays to this due to the bag going up from 50 to 80 bucks, needless to say that was a mistake, not only did the dog food smell awful upon opening the bag, especially compared to Rachael rays since that food had no smell nor strong odor like this one, after a couple weeks our dogs would eat the food less and less and started throwing it up, what trash, the food smelled like vomit and it made our dogs vomit, do not recommend, won’t be buying again, I guess you get what you pay for

I really don’t know why but my dogs will not eat this. I will not order again. So disappointed

I’m unsure what’s with this product but I purchased the variety pack and my dog was sick and puked all over the house for days immediately after eating it.

I’ve given my small dog to is food before and she’s been totally ok with it. This was the first time I ordered it off of Amazon as I was having a hard time finding wet food in stores. She ate it fine but has had horrible diarrhea from this particular batch. I have given her one of each flavor over the past 4 days but all of them are making her sick. I will not be giving her anymore and throwing the other 20 out. I’m not sure what is wrong with this batch as it doesn’t expire until May of 2024.

The ingredients in the food is pure trash. Corn being the largest percentage, followed by a list of fillers and bone meal which is not what you’d expect out of “chicken” dog food. The vegetables in the entire bag are less than 1% as they are listed after salt which by law cannot have more than 1%. So roasted vegetables not so much. I think that the pictures on the front make you think your getting some delicious food for your dog when your not. Not to mention the Blues & yellows mentioned which is basically paint to make you that those are the vegetable pieces.
As far as value your getting a 33 pound bag for under 20$ so I’d say that’s a value, however with the amount of corn that’s in this food it’s probably worth more like 8-10$.
Presentation is deceiving if your not a ingredient list junkie like I am.
I purchased the food solely as a back up when I run out of the everyday dog food because with 3 dogs it happens. I feel guilty for giving them garbage. But I will add some REAL vegetables to the bowl and supplement with some form of protein if need be.
I would NOT recommend anyone feed their pet this in the long term. There are other dog food that have much more quality, your going to spend more but at least your not giving your dog corn, paint & ground up bones.


Switched brands and dog started developing sores and spots of hair loss. Was advised pedigree is one of the worst. Don't understand why any vet would recommend besides a marketing ploy. Learning more about the ingredients used all I can recommend is stay away

My dog will not eat this ?

DO NOT BUY THIS! If I could give less than 1 star I would.

I don't think I've ever written a review, but this dog food was so terrible I feel obligated to warn other dog owners. Are usually by Rachael Ray's dog food, which is another cheap brand on Amazon, but I was ridiculously broke this month and thought one month of pedigree would be OK. I was very very wrong. Within starting to feed my one-year-old golden retriever at this dog food, he immediately in the following mornings would wake up with awful yellow squirting diarrhea that he couldn't contain. Along with several nights of this, he would also wake up and throw up yellow mucus, nothing else coming out but clearly his stomach was revolting whatever was in it. I did some research on pedigree online to look at unbiased customer reviews, and was amazed to find the Pedegree has less than one star and has been known to make dogs sick and even kill them. I'm absolutely horrified that such a dog food could exist on Amazon with such high ratings when I have absolutely no doubt that pedigree dog food was the cause of my dog sickness the past month. Coming from a very broke college student with the dog who eats more than I can handle, do not buy this dog food. If you need cheap, look elsewhere. I will never buy pedigree again and I feel absolutely awful for making my dog go through whatever poison this was

This product has become too expensive! My Dog refuses to eat the filet and is not fond of the beef and noodles,so that over half the food that is wasted. I want to discontinue my subscription !

My dog has eaten this same food for nearly a year, prior to that it was the puppy version. Decided since price was equal to in store, to order it online. Took a picture of an ad on the bag I hadn’t seen before then tossed the bag once the food was in the container. Right away the kibbles didn’t seem correct, but brands change things up. My dog will eat anything except this bag of kibble. I have to throw cooked rice, meat, etc in it for her to try and eat. It’s not the kibble in general, she at her stash at grandmas no problem (from another bag).
Looked up the ad and it has since ended. Not sure how long dog food stays fresh, but my dog has declared it inedible. For her safety I decided to throw it out. Unfortunate, might not risk getting another bag.
"""

In [13]:
def split_reviews(text):
    sentences = nltk.sent_tokenize(text)
    
    sentences = [sentence.split('\n') for sentence in sentences]
    sentences = [sentence for sublist in sentences for sentence in sublist]
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    return sentences


In [17]:
pos_reviews = split_reviews(pos_review_cluster)

In [19]:
neg_reviews = split_reviews(neg_review_cluster)

In [79]:
def bert_summarize(text, num_sentences=3):
    text = "".join(text)
    from summarizer import Summarizer

    model = Summarizer()


    summary = model(text, num_sentences=num_sentences)
    return summary

In [80]:
bert_summarize(pos_reviews)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


"Great dog food, my dog has a very sensitive stomach and after spending loads of money on food this one does not affect her stomach and it is a great price!I only buy Pedigree for my dog Apollo. I like that more than one flavor is included in the box. My dog gets half a pouch every morning with a half a pouch of moist in meaty mixed in and has no problems with this food she can't wait to get it of course I have to wake her up first never goes to wasteFav dog food so far, been having both my dogs eat it for about 5 years nowJust that my dogs loved itMy 15 yr Pom mix loves this high protein food."

In [81]:
bert_summarize(neg_reviews)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


'Bought 2 bags of the 18lb Grilled Steak & Vegetables and had to throw them away because my dog had an allergic reaction. Not to mention the Blues & yellows mentioned which is basically paint to make you that those are the vegetable pieces. I will never buy pedigree again and I feel absolutely awful for making my dog go through whatever poison this wasThis product has become too expensive!My Dog refuses to eat the filet and is not fond of the beef and noodles,so that over half the food that is wasted.'

In [39]:
! wget http://nlp.stanford.edu/data/glove.6B.zip
! unzip glove*.zip

--2023-11-29 05:02:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-29 05:02:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-29 05:02:25--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [82]:
def text_rank_summarize(sentences, num_sentences=4, embd_size = 300):

    word_embeddings = {}
    f = open("glove.6B." + str(embd_size) + "d.txt", encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()


    clean_sentences = [s.replace("[^a-zA-Z]", " ") for s in sentences]
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((embd_size,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((embd_size,))
        sentence_vectors.append(v)
        
    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,embd_size), sentence_vectors[j].reshape(1,embd_size))[0,0]

    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


    important_sentences = []
    for i in range(num_sentences):
        important_sentences.append(ranked_sentences[i][1])
    
    return important_sentences


In [83]:
text_rank_summarize(pos_reviews)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['We have 4 large dogs and they go through dog food like crazy!',
 'Great dog food, my dog has a very sensitive stomach and after spending loads of money on food this one does not affect her stomach and it is a great price!',
 'My dog loves eating the same thing, every day.',
 "My dog gets half a pouch every morning with a half a pouch of moist in meaty mixed in and has no problems with this food she can't wait to get it of course I have to wake her up first never goes to waste"]

In [84]:
text_rank_summarize(neg_reviews)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Switched from Rachael Rays to this due to the bag going up from 50 to 80 bucks, needless to say that was a mistake, not only did the dog food smell awful upon opening the bag, especially compared to Rachael rays since that food had no smell nor strong odor like this one, after a couple weeks our dogs would eat the food less and less and started throwing it up, what trash, the food smelled like vomit and it made our dogs vomit, do not recommend, won’t be buying again, I guess you get what you pay for',
 'There are other dog food that have much more quality, your going to spend more but at least your not giving your dog corn, paint & ground up bones.',
 'I think that the pictures on the front make you think your getting some delicious food for your dog when your not.',
 "Are usually by Rachael Ray's dog food, which is another cheap brand on Amazon, but I was ridiculously broke this month and thought one month of pedigree would be OK."]

# Clustering reviews by different types:

In [1]:
sentences = [
    "This is my cat's favorite meal! My cat really likes the flavor, he always prefers Salmon.", #flavor
    "This cat food is worth the money for the low price and I love the product.", #value
    "This cat food is super cheap while at the same time gives a good amount for not being too expensive!", #value
    "I have never bought another type of cat food where my cat's reaction the aroma and taste is this strong. My cat smells it an instantly runs over!", #flavor
    "Both my cats love how the food tastes, they always meow for more!", #flavor
    "Wow, talk about value! 16 packs for only 10 dollars is a good deal!", #value
    "My vet recommended me this pet food since it is high in nutrition and vitamins. Perfect meal for my cats.", #nutritious
    "Healthy for my cats, has all the important parts of a meal!", #nutritious,
    "My cat ate some last night and threw up, is this even safe??", #bad
    "Had to take my cat to the hospital after she ate this! Do NOT recommend at all!," #bad
    "Save your money, there's plenty of other better brands and products than this.", #bad
    "On weekends, I like to take my dog for a walk.", #random
]
num_clusters = 5 #flavor, value, nutritious, bad, random

In [2]:
from transformers import BertModel, BertTokenizer
from sklearn.cluster import KMeans
import numpy as np

def cluster_sentences_bert_kmeans(sentences, num_clusters):
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    tokenized_sentences = [tokenizer(sentence, return_tensors='pt') for sentence in sentences]
    embeddings = [model(**tokens).last_hidden_state.mean(dim=1).detach().numpy().squeeze() for tokens in tokenized_sentences]

    embeddings_array = np.array(embeddings)

    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings_array)

    clusters = [[] for _ in range(num_clusters)]
    for i, label in enumerate(labels):
        clusters[label].append(sentences[i])

    return clusters


result_clusters = cluster_sentences_bert_kmeans(sentences, num_clusters)

for cluster in result_clusters:
    for s in cluster:
        print(s, '\n')
    print("\n**********\n")


This cat food is super cheap while at the same time gives a good amount for not being too expensive! 


**********

This cat food is worth the money for the low price and I love the product. 

I have never bought another type of cat food where my cat's reaction the aroma and taste is this strong. My cat smells it an instantly runs over! 

My vet recommended me this pet food since it is high in nutrition and vitamins. Perfect meal for my cats. 

Healthy for my cats, has all the important parts of a meal! 

Had to take my cat to the hospital after she ate this! Do NOT recommend at all!,Save your money, there's plenty of other better brands and products than this. 


**********

On weekends, I like to take my dog for a walk. 


**********

Wow, talk about value! 16 packs for only 10 dollars is a good deal! 


**********

This is my cat's favorite meal! My cat really likes the flavor, he always prefers Salmon. 

Both my cats love how the food tastes, they always meow for more! 

My cat ate

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
from transformers import BertModel, BertTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

def cluster_sentences_lda(sentences, num_clusters):
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    tokenized_sentences = [tokenizer(sentence, return_tensors='pt') for sentence in sentences]
    embeddings = [model(**tokens).last_hidden_state.mean(dim=1).detach().numpy().squeeze() for tokens in tokenized_sentences]

    embeddings_array = np.array(embeddings)

    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(sentences)

    lda = LatentDirichletAllocation(n_components=num_clusters, random_state=42)
    topic_assignments = lda.fit_transform(X)

    clusters = [[] for _ in range(num_clusters)]
    for i, topic_probs in enumerate(topic_assignments):
        dominant_topic = np.argmax(topic_probs)
        clusters[dominant_topic].append(sentences[i])

    return clusters

result_clusters = cluster_sentences_lda(sentences, num_clusters)

for cluster in result_clusters:
    for s in cluster:
        print(s, '\n')
    print("\n**********\n")

Both my cats love how the food tastes, they always meow for more! 

Wow, talk about value! 16 packs for only 10 dollars is a good deal! 


**********

This cat food is worth the money for the low price and I love the product. 

My cat ate some last night and threw up, is this even safe?? 

Had to take my cat to the hospital after she ate this! Do NOT recommend at all!,Save your money, there's plenty of other better brands and products than this. 

On weekends, I like to take my dog for a walk. 


**********

This is my cat's favorite meal! My cat really likes the flavor, he always prefers Salmon. 

I have never bought another type of cat food where my cat's reaction the aroma and taste is this strong. My cat smells it an instantly runs over! 

My vet recommended me this pet food since it is high in nutrition and vitamins. Perfect meal for my cats. 

Healthy for my cats, has all the important parts of a meal! 


**********


**********

This cat food is super cheap while at the same tim

In [5]:
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

def cluster_sentences_gpt2_kmeans(sentences, num_clusters):
    model_name = 'gpt2'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2Model.from_pretrained(model_name)

    tokenized_sentences = [tokenizer(sentence, return_tensors='pt') for sentence in sentences]
    embeddings = [model(**tokens).last_hidden_state.mean(dim=1).detach().numpy().squeeze() for tokens in tokenized_sentences]

    embeddings_array = np.array(embeddings)

    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings_array)

    clusters = [[] for _ in range(num_clusters)]
    for i, label in enumerate(labels):
        clusters[label].append(sentences[i])

    return clusters


result_clusters = cluster_sentences_gpt2_kmeans(sentences, num_clusters)

for cluster in result_clusters:
    for s in cluster:
        print(s, '\n')
    print("\n**********\n")

This cat food is super cheap while at the same time gives a good amount for not being too expensive! 

Healthy for my cats, has all the important parts of a meal! 

My cat ate some last night and threw up, is this even safe?? 


**********

This is my cat's favorite meal! My cat really likes the flavor, he always prefers Salmon. 

Both my cats love how the food tastes, they always meow for more! 

My vet recommended me this pet food since it is high in nutrition and vitamins. Perfect meal for my cats. 


**********

Wow, talk about value! 16 packs for only 10 dollars is a good deal! 


**********

This cat food is worth the money for the low price and I love the product. 

I have never bought another type of cat food where my cat's reaction the aroma and taste is this strong. My cat smells it an instantly runs over! 

Had to take my cat to the hospital after she ate this! Do NOT recommend at all!,Save your money, there's plenty of other better brands and products than this. 


********

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
